In [2]:
# IMPORTS
import os
import nltk
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

# Obtenir le chemin absolu du répertoire de données NLTK
# nltk_data_dir = os.path.abspath('../.venv/nltk_data')

# # Définir la variable d'environnement NLTK_DATA
# os.environ['NLTK_DATA'] = nltk_data_dir

# # Télécharger les ressources NLTK
# nltk.download('punkt', download_dir=nltk_data_dir)
# nltk.download('stopwords', download_dir=nltk_data_dir)
# nltk.download('wordnet', download_dir=nltk_data_dir)
# nltk.download('punkt_tab', download_dir=nltk_data_dir)

CORPUS FONCTION de l'APP

In [3]:
import speech_recognition as sr

# Fonction pour convertir l'audio en texte
# Cette fonction utilise un microphone pour capturer l'audio en temps réel, puis le transforme en texte 
# en utilisant le service de reconnaissance vocale Google.
def audio_to_text():
    conv = []  # Liste pour stocker les textes reconnus
    recognizer = sr.Recognizer()  # Initialisation de l'objet de reconnaissance vocale
    
    # Capture de l'audio à partir du microphone
    with sr.Microphone() as source:
        print("En écoute...")
        audio = recognizer.listen(source)  # Écoute de l'audio depuis la source
        print("Écoute terminée.")

    try:  
        # Conversion de l'audio en texte
        text = recognizer.recognize_google(audio)
        conv.append(text)
    except sr.UnknownValueError:
        print("Désolé, je n'ai pas compris.")
    
    return conv

# Fonction pour prétraiter les données textuelles
# Cette fonction nettoie et transforme un ensemble de texte brut pour une meilleure analyse ou modélisation.
def preprocessing(dataset, num_of_rows=1):
    stemmer = WordNetLemmatizer()  # Initialisation du lemmatiseur pour la réduction des mots à leur forme de base
    corpus = []  # Liste pour stocker les textes nettoyés

    for i in range(0, num_of_rows):
        # Suppression des caractères spéciaux
        document = re.sub(r'\W', ' ', dataset[i])

        # Suppression des caractères isolés dans le texte
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Suppression des caractères isolés en début de ligne
        document = re.sub(r'^\s*[a-zA-Z]\s+', ' ', document)

        # Remplacement des espaces multiples par un seul espace
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Conversion du texte en minuscules
        document = document.lower()

        # Séparation des mots
        document = document.split()

        # Lemmatisation des mots pour les réduire à leur forme de base
        document = [stemmer.lemmatize(word) for word in document]

        # Reconstruction du texte prétraité
        document = ' '.join(document)

        # Ajout du texte nettoyé au corpus
        corpus.append(document)

    return corpus

# Méthode pour prédire avec un modèle utilisant CountVectorizer
# Cette fonction divise les données en ensembles d'entraînement et de test, puis évalue un modèle de classification Naive Bayes.
def predict_using_count_vectoriser(dataset, num_of_rows, callData):
    # Prétraitement des données textuelles
    x = preprocessing(dataset['call_content'], num_of_rows)
    y = dataset.Label

    # Division des données en ensembles d'entraînement (70%) et de test (30%)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

    # Prétraitement des nouvelles données pour le test en direct
    live_test = preprocessing(callData, len(callData))

    # Vectorisation des textes : comptage des occurrences de chaque mot avec CountVectorizer
    count_vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))

    # Ajustement du vectoriseur aux données d'entraînement et transformation en matrices
    count_train = count_vectorizer.fit_transform(x_train)
    count_test = count_vectorizer.transform(x_test)
    live_count_test = count_vectorizer.transform(live_test)

    # Entraînement du modèle Naive Bayes multinomial
    nbclassifier = MultinomialNB()
    nbclassifier.fit(count_train, y_train)

    # Prédictions sur l'ensemble de test et les données en direct
    y_pred = nbclassifier.predict(count_test)
    live_y_pred = nbclassifier.predict(live_count_test)

    # Calcul de la précision et de la matrice de confusion
    score = metrics.accuracy_score(y_test, y_pred)
    cm = metrics.confusion_matrix(y_pred, y_test, labels=['normal', 'fraud'])

    print("Score de précision avec CountVectorizer :", score)
    print("MATRICE DE CONFUSION\n", cm)

    return callData, live_y_pred

# Méthode pour prédire avec un modèle utilisant TfidfVectorizer
# Cette fonction divise les données en ensembles d'entraînement et de test, puis évalue un modèle de classification Naive Bayes.
def predict_using_tfidf_vectoriser(dataset, num_of_rows, callData):
    # Prétraitement des données textuelles
    x = preprocessing(dataset['call_content'], num_of_rows)
    y = dataset.Label

    # Division des données en ensembles d'entraînement (70%) et de test (30%)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

    # Prétraitement des nouvelles données pour le test en direct
    live_x_test = preprocessing(callData, len(callData))

    # Vectorisation des textes : importance pondérée des mots avec TfidfVectorizer
    tfidfvectorizer = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))

    # Ajustement du vectoriseur aux données d'entraînement et transformation en matrices
    tfidf_train = tfidfvectorizer.fit_transform(x_train)
    tfidf_test = tfidfvectorizer.transform(x_test)
    live_tfidf_test = tfidfvectorizer.transform(live_x_test)

    # Entraînement du modèle Naive Bayes multinomial
    nbclassifier = MultinomialNB()
    nbclassifier.fit(tfidf_train, y_train)

    # Prédictions sur l'ensemble de test et les données en direct
    y_predict = nbclassifier.predict(tfidf_test)
    live_y_predict = nbclassifier.predict(live_tfidf_test)

    # Calcul de la précision et de la matrice de confusion
    score = metrics.accuracy_score(y_test, y_predict)
    cm = metrics.confusion_matrix(y_predict, y_test, labels=['normal', 'fraud'])

    print("Score de précision avec TfidfVectorizer :", score)
    print("MATRICE DE CONFUSION\n", cm)

    return callData, live_y_predict

PRE-TRAITEMENT/TRAITEMENT (à la mano)

In [5]:
# Chargement DATA
dataset=pd.read_csv(r"../data/raw/Fraud_calls.txt",sep='|')
# dataset.head()

# Nettoyage DATA
num_of_rows,y=dataset.shape
callData='Bonjour, je vends des Tesla à 200 euros la voiture, tu en veux ?'

PREDICTION MODEL 1 (countvecto)

In [7]:
# Entraînement sur le dataset Fraud_calls.txt
print("\n ********** MODELE COUNT VECTORISER **********\n")
conversation,label=predict_using_count_vectoriser(dataset,num_of_rows,callData)
print("Texte Reçu:{} \npredicted as label:{}".format(conversation,label))

print("\n ********** MODELE TGIDF VECTORISER **********\n")
conversation,label=predict_using_tfidf_vectoriser(dataset,num_of_rows,callData)
print("Texte Reçu:{}  \npredicted as label:{}".format(conversation,label))


 ********** MODELE COUNT VECTORISER **********

Score de précision avec CountVectorizer : 0.9735658042744657
MATRICE DE CONFUSION
 [[1554   23]
 [  24  177]]
Texte Reçu:Bonjour, je vends des Tesla à 200 euros la voiture, tu en veux ? 
predicted as label:['normal' 'normal' 'normal' 'normal' 'normal' 'normal' 'normal' 'normal'
 'normal' 'normal' 'normal' 'normal' 'normal' 'normal' 'normal' 'normal'
 'normal' 'normal' 'normal' 'normal' 'normal' 'normal' 'normal' 'normal'
 'normal' 'normal' 'normal' 'normal' 'normal' 'normal' 'normal' 'normal'
 'normal' 'normal' 'normal' 'normal' 'normal' 'normal' 'normal' 'normal'
 'normal' 'normal' 'normal' 'normal' 'normal' 'normal' 'normal' 'normal'
 'normal' 'normal' 'normal' 'normal' 'normal' 'normal' 'normal' 'normal'
 'normal' 'normal' 'normal' 'normal' 'normal' 'normal' 'normal' 'normal']

 ********** MODELE TGIDF VECTORISER **********

Score de précision avec TfidfVectorizer : 0.9780652418447694
MATRICE DE CONFUSION
 [[1575   36]
 [   3  164]]
T